<a href="https://colab.research.google.com/github/ssvadla/Research_01/blob/main/Copy_of_EM_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from google.colab import drive

drive.mount('/content/drive')

train1 = pd.read_csv('/content/drive/My Drive/Research/train_data1.csv')
train2 = pd.read_csv('/content/drive/My Drive/Research/train_data2.csv')
train3 = pd.read_csv('/content/drive/My Drive/Research/train_data3.csv')
train4 = pd.read_csv('/content/drive/My Drive/Research/train_data4.csv')
train5 = pd.read_csv('/content/drive/My Drive/Research/train_data5.csv')
train6 = pd.read_csv('/content/drive/My Drive/Research/train_data6.csv')
train7 = pd.read_csv('/content/drive/My Drive/Research/train_data7.csv')
train8 = pd.read_csv('/content/drive/My Drive/Research/train_data8.csv')
train9 = pd.read_csv('/content/drive/My Drive/Research/train_data9.csv')
train10 = pd.read_csv('/content/drive/My Drive/Research/train_data10.csv')
train_highKappa = pd.read_csv('/content/drive/My Drive/Research/train_data_highkappa.csv')
train1.head()

train = train1
train_list = [train2,train3,train4,train5,train6,train7,train8,train9,train10,train_highKappa]
for i in train_list:
  #print(i)
  train = train.append(i)

train.sort_values("Sentence", inplace = True)
print(len(train))
new_train = train.drop_duplicates(subset ="Sentence")
train = new_train
train['Target']=train['Target'].replace(['Others'],'Invalid')
train['Target'].unique()

import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

len(train['Sentence'])
text = clean_text(train['Sentence'])

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)
X_tfidf = tfidf_vect.fit_transform(train['Sentence'])
print(X_tfidf.shape)

test = pd.read_csv(r'/content/drive/My Drive/Research/test_data.csv')

test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))
t_p = tfidf_vect.transform(test['Sentence'])



unlabel = pd.read_csv(r'/content/drive/My Drive/Research/Unlabeled_data.csv')
#unlabel.head()

del unlabel['Complete']
del unlabel['Unnamed: 0']

unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
unlabel['text'] = unlabel['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join(x for x in x.split() if x not in words))


from textblob import TextBlob
from textblob import Word
nltk.download('wordnet')
nltk.download('punkt')
unlabel['text'] = unlabel['text'].apply(lambda x: TextBlob(x).words)
unlabel['text'] = unlabel['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

unlabel_1 = unlabel.loc[:500]


def index_reset(unlabel_2):
  unlabel_2.reset_index(inplace=True)
  del unlabel_2['index']
  #print(unlabel_2.head())
  return unlabel_2

unlabel_1 = index_reset(unlabel_1)
unlabel_1_copy = unlabel_1



x_un1 = tfidf_vect.transform(unlabel_1['text'])

unlabel_1['Target']=-1

train = train.rename(columns={'Sentence':'text'})
from sklearn.preprocessing import LabelEncoder
train['Target']= LabelEncoder().fit_transform(train['Target'])

train_and_unlabel =  pd.concat([train,unlabel_1])
print(len(train))
print(len(unlabel_1))
print(len(train_and_unlabel))

train_and_unlabel.reset_index(inplace=True)
del train_and_unlabel['index']
del train_and_unlabel['Unnamed: 0']


train_and_unlabel_words = clean_text(train_and_unlabel['text'])
train_and_unlabel_words_wo_duplicates = list(set(train_and_unlabel_words))
train_words = clean_text(train['text'])
train_words_wo_duplicates = list(set(train_words))
unlabel_words = clean_text(unlabel_1['text'])
unlabel_words_wo_duplicates = list(set(unlabel_words))


PF = []
NF = []
Threshold_feature = 1

def feature_set_selection(train_and_unlabel_words_wo_duplicates,train_words,unlabel_words):
  for term in train_and_unlabel_words_wo_duplicates:
    print(term)
    freq_train = train_words.count(term) / len(train_words)
    print(freq_train)
    freq_unlabel = unlabel_words.count(term) / len(unlabel_words)
    print(freq_unlabel)
    if freq_unlabel == 0:
      PF.append(term)
    elif (freq_train // freq_unlabel) > Threshold_feature:
      PF.append(term)
    else:
      NF.append(term)


feature_set_selection(train_and_unlabel_words_wo_duplicates,train_words,unlabel_words)




def RN_selection(RN, unlabel_1):
  iteration_RN = 0
  RN_to_be_removed = []
  pos_to_be_removed = []
  freq_each_word_list = []
  count = 0

  for doc in unlabel_1['text']:
    print("iteration_RN",iteration_RN)
    print(doc)
    doc_words = clean_text(doc)
    doc_words_wo_duplicates = list(set(doc_words))
    for each_doc_word in doc_words_wo_duplicates:
      print(each_doc_word)
      freq_each_doc_word = doc_words.count(each_doc_word)
      freq_each_word_list.append(freq_each_doc_word)
      print(freq_each_doc_word)
      print(each_doc_word in PF)
      if (freq_each_doc_word > 0) and (each_doc_word in PF):
        print("Yes writing ........")
        pos_to_be_removed.append(count)
        RN_to_be_removed.append(doc)
        print("breaking:::::::::::")
        break
    count = count + 1
    iteration_RN = iteration_RN + 1
  
  RN.drop(pos_to_be_removed,axis=0,inplace=True)
  #Q =  unlabel_1.loc[pos_to_be_removed,:]
  #print(len(Q))
  return RN,pos_to_be_removed

unlabel_2 = unlabel_1[0:10] #comment it 
RN = unlabel_2 #comment
RN,pos_to_be_removed = RN_selection(RN,unlabel_2) # comment

Q =  unlabel_1.loc[pos_to_be_removed,:]


import lightgbm as lgb

def classifier_select(train, Q, RN):

  loop_variable =0
  #for i in range(0,3):
  while(1):  # comment it change the range 
    print(loop_variable)
    p_and_RN = pd.concat([train,RN])
    print("#######target unique",np.unique(p_and_RN['Target']))
    p_and_RN.reset_index(inplace=True,drop=True)
    p_and_RN_vect = tfidf_vect.transform(p_and_RN['text'])
    p_and_RN_vect_df=pd.DataFrame(p_and_RN_vect.toarray())

    Q_vect = tfidf_vect.transform(Q['text'])
    Q_vect_df=pd.DataFrame(Q_vect.toarray())

    lgb_classifier = lgb.LGBMClassifier()
    lgb_classifier.fit(p_and_RN_vect_df, p_and_RN['Target'])
    np.unique(p_and_RN['Target'])

    #checking the classifier if it gives best results
    train_vect = tfidf_vect.transform(train['text'])
    train_vect_df=pd.DataFrame(train_vect.toarray())
    train_pred = lgb_classifier.predict(train_vect_df)
    print("#############train pred", np.unique(train_pred))
    classified_negative = (train_pred.tolist()).count(-1)
    percentage = (classified_negative / len(train_pred)) * 100
    model_list.append(lgb_classifier)
    if percentage < percent_thresh:
      #pass
      thresh_model_list.append(lgb_classifier)
      

    Q_pred = lgb_classifier.predict(Q_vect_df)
    np.unique(Q_pred)

    

    count_q = 0
    total_q = 0 
    out_pos_q = []
    pos_q = []
    for i in Q_pred:
      #print(i)
      if i == -1:
        pos_q.append(count_q)
        total_q = total_q + 1
      else:
        out_pos_q.append(count_q)

      count_q = count_q + 1

    Q.reset_index(inplace=True,drop=True)
    W = Q.loc[pos_q,:]


    if W.empty :
      print("W is empty, came out of loop")
      break
    else:
      Q_new = Q.loc[out_pos_q,:]
      Q = Q_new
      RN = pd.concat([RN,W])
      RN.reset_index(inplace=True,drop=True)
      loop_variable = loop_variable + 1
    print("completed iteration")
    

  return lgb_classifier, model_list , thresh_model_list

percent_thresh = 5
model_list = []
thresh_model_list = []
lgb_classifier, op_model_list, op_thresh_model_list = classifier_select(train, Q, RN)

from sklearn.metrics import classification_report

for model in op_model_list:
  test_pred = model.predict(t_p.toarray())
  test['Target']= LabelEncoder().fit_transform(test['Target'])
  classification_report_test = classification_report(test['Target'],test_pred,digits=4)
  print(classification_report_test)

for model in op_thresh_model_list:
  test_pred = model.predict(t_p.toarray())
  test['Target']= LabelEncoder().fit_transform(test['Target'])
  classification_report_test = classification_report(test['Target'],test_pred,digits=4)
  print(classification_report_test)

  



Mounted at /content/drive


,Unnamed: 0,Sentence,Target
0,659,Appellant had stated to the officers that she ...,Invalid
1,3456,We shall discuss the facts more fully in conne...,Others
2,2043,"â€œPerjury is a false statement, either writte...",Invalid
3,3344,The offense is felony theft by false pretext; ...,Issue
4,3231,Numerous contentions urging the commission of ...,Issue


In [ ]:
from sklearn.metrics import classification_report

for model in op_model_list:
  test_pred = model.predict(t_p.toarray())
  test['Target']= LabelEncoder().fit_transform(test['Target'])
  classification_report_test = classification_report(test['Target'],test_pred,digits=4)
  print(classification_report_test)


              precision    recall  f1-score   support

           0     0.3922    0.2597    0.3125        77
           1     0.6957    0.6154    0.6531        26
           2     0.6743    0.8839    0.7650       267
           3     0.5263    0.3614    0.4286        83
           4     0.9167    0.3235    0.4783        34
           5     0.5714    0.4706    0.5161        34

    accuracy                         0.6315       521
   macro avg     0.6294    0.4858    0.5256       521
weighted avg     0.6192    0.6315    0.6040       521



In [ ]:
for model in op_thresh_model_list:
  test_pred = model.predict(t_p.toarray())
  test['Target']= LabelEncoder().fit_transform(test['Target'])
  classification_report_test = classification_report(test['Target'],test_pred,digits=4)
  print(classification_report_test)

              precision    recall  f1-score   support

           0     0.3922    0.2597    0.3125        77
           1     0.6957    0.6154    0.6531        26
           2     0.6743    0.8839    0.7650       267
           3     0.5263    0.3614    0.4286        83
           4     0.9167    0.3235    0.4783        34
           5     0.5714    0.4706    0.5161        34

    accuracy                         0.6315       521
   macro avg     0.6294    0.4858    0.5256       521
weighted avg     0.6192    0.6315    0.6040       521



In [ ]:
np.unique(RN['Target'])

In [ ]:
test_pred =  lgb_classifier.predict(t_p.toarray())

In [ ]:
from sklearn.metrics import classification_report
test['Target']= LabelEncoder().fit_transform(test['Target'])
classification_report_test = classification_report(test['Target'],test_pred,digits=4)
print(classification_report_test)